In [2]:
# Read, stack, clean, and save aew1 and aew2 JSON data (2023-2025), display info

# Import necessary libraries
import pandas as pd
import glob
import json
import numpy as np
import os

# Define directories
extract_dir = "../Extracted_Dataset"
processed_dir = "../Processed_Data"

# Create Processed_Data directory if it doesn't exist
os.makedirs(processed_dir, exist_ok=True)

# Function to read and process a single JSON file
def process_json(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        # Expand 'data' column if it exists (nested dicts with 'time', 'E1', etc.)
        if 'data' in df.columns:
            expanded_data = pd.json_normalize(df['data'])
            df = df.drop('data', axis=1).join(expanded_data)
        # Add missing E13 and E14 columns with constant values if not present
        if 'E13' not in df.columns:
            df['E13'] = 1313
        if 'E14' not in df.columns:
            df['E14'] = 1414
        # Convert 'time' to datetime if present
        if 'time' in df.columns:
            df['time'] = pd.to_datetime(df['time'], errors='coerce')
        return df
    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {e}")
        return None

# 1. Process aew1 JSON files (2023-2025)
aew1_path = os.path.join(extract_dir, 'EDHD_Ch9_Dataset/edhd-optima-dataset/aew1')
aew1_json_files = sorted(glob.glob(os.path.join(aew1_path, '*.json')))
aew1_dfs = []
for f in aew1_json_files:
    df = process_json(f)
    if df is not None:
        aew1_dfs.append(df)
if aew1_dfs:
    aew1_raw_timeseries = pd.concat(aew1_dfs, ignore_index=True)
    # Sort by time to make continuous timeseries
    if 'time' in aew1_raw_timeseries.columns:
        aew1_raw_timeseries = aew1_raw_timeseries.sort_values('time').reset_index(drop=True)
    # Save raw timeseries
    aew1_raw_timeseries.to_csv(os.path.join(processed_dir, 'aew1_raw_timeseries.csv'), index=False)
    print("aew1_raw_timeseries created with shape:", aew1_raw_timeseries.shape)
    print("Saved to:", os.path.join(processed_dir, 'aew1_raw_timeseries.csv'))
else:
    print("No valid aew1 JSON files processed (2023-2025).")
    aew1_raw_timeseries = pd.DataFrame()

# 2. Process aew2 JSON files (2023-2025)
aew2_path = os.path.join(extract_dir, 'EDHD_Ch9_Dataset/edhd-optima-dataset/aew2')
aew2_json_files = sorted(glob.glob(os.path.join(aew2_path, '*.json')))
aew2_dfs = []
for f in aew2_json_files:
    df = process_json(f)
    if df is not None:
        aew2_dfs.append(df)
if aew2_dfs:
    aew2_raw_timeseries = pd.concat(aew2_dfs, ignore_index=True)
    # Sort by time to make continuous timeseries
    if 'time' in aew2_raw_timeseries.columns:
        aew2_raw_timeseries = aew2_raw_timeseries.sort_values('time').reset_index(drop=True)
    # Save raw timeseries
    aew2_raw_timeseries.to_csv(os.path.join(processed_dir, 'aew2_raw_timeseries.csv'), index=False)
    print("aew2_raw_timeseries created with shape:", aew2_raw_timeseries.shape)
    print("Saved to:", os.path.join(processed_dir, 'aew2_raw_timeseries.csv'))
else:
    print("No valid aew2 JSON files processed (2023-2025).")
    aew2_raw_timeseries = pd.DataFrame()

# Function to clean the raw timeseries
def clean_timeseries(df):
    # Define expected numeric columns
    numeric_cols = ['E1', 'E3', 'E5', 'E13', 'E14', 'T2', 'T5', 'T7', 'T10', 'T12', 'T13', 'WEATHER_TEMP', 'WEATHER_DEWPOINT', 'WEATHER_WINDSPEED', 'WEATHER_ISDAYTIME']
    # Convert numeric columns, coerce errors to NaN
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    # Drop rows with invalid time
    if 'time' in df.columns:
        df = df.dropna(subset=['time'])
    # Drop fully NaN rows
    df = df.dropna(how='all')
    # Fill remaining NaNs with 0
    df = df.fillna(0)
    return df

# Clean aew1_raw_timeseries
if not aew1_raw_timeseries.empty:
    aew1_clean_timeseries = clean_timeseries(aew1_raw_timeseries)
    aew1_clean_timeseries.to_csv(os.path.join(processed_dir, 'aew1_clean_timeseries.csv'), index=False)
    print("aew1_clean_timeseries created with shape:", aew1_clean_timeseries.shape)
    print("Saved to:", os.path.join(processed_dir, 'aew1_clean_timeseries.csv'))
else:
    print("No aew1_raw_timeseries to clean.")
    aew1_clean_timeseries = pd.DataFrame()

# Clean aew2_raw_timeseries
if not aew2_raw_timeseries.empty:
    aew2_clean_timeseries = clean_timeseries(aew2_raw_timeseries)
    aew2_clean_timeseries.to_csv(os.path.join(processed_dir, 'aew2_clean_timeseries.csv'), index=False)
    print("aew2_clean_timeseries created with shape:", aew2_clean_timeseries.shape)
    print("Saved to:", os.path.join(processed_dir, 'aew2_clean_timeseries.csv'))
else:
    print("No aew2_raw_timeseries to clean.")
    aew2_clean_timeseries = pd.DataFrame()

# Display column names for clean timeseries data
if not aew1_clean_timeseries.empty:
    print("\nColumn names in aew1_clean_timeseries:")
    print(aew1_clean_timeseries.columns.tolist())
else:
    print("\nNo columns for aew1_clean_timeseries (empty).")

if not aew2_clean_timeseries.empty:
    print("\nColumn names in aew2_clean_timeseries:")
    print(aew2_clean_timeseries.columns.tolist())
else:
    print("\nNo columns for aew2_clean_timeseries (empty).")


aew1_raw_timeseries created with shape: (67545, 20)
Saved to: ../Processed_Data/aew1_raw_timeseries.csv
aew2_raw_timeseries created with shape: (70043, 20)
Saved to: ../Processed_Data/aew2_raw_timeseries.csv
aew1_clean_timeseries created with shape: (67545, 20)
Saved to: ../Processed_Data/aew1_clean_timeseries.csv
aew2_clean_timeseries created with shape: (70043, 20)
Saved to: ../Processed_Data/aew2_clean_timeseries.csv

Column names in aew1_clean_timeseries:
['interval', 'start', 'end', 'boxId', 'time', 'E1', 'E3', 'E5', 'T10', 'T12', 'T13', 'T2', 'T5', 'T7', 'WEATHER_DEWPOINT', 'WEATHER_TEMP', 'WEATHER_WINDSPEED', 'WEATHER_ISDAYTIME', 'E13', 'E14']

Column names in aew2_clean_timeseries:
['interval', 'start', 'end', 'boxId', 'time', 'E1', 'E13', 'E14', 'E3', 'E5', 'T10', 'T12', 'T13', 'T2', 'T5', 'T7', 'WEATHER_DEWPOINT', 'WEATHER_TEMP', 'WEATHER_WINDSPEED', 'WEATHER_ISDAYTIME']


In [2]:
# Read, stack, clean, and save Energy Overview XLS data (2009-2025), display info

# Import necessary libraries
import pandas as pd
import glob
import os
import numpy as np

# Define directories
extract_dir = "../Extracted_Dataset"
processed_dir = "../Processed_Data"

# Create Processed_Data directory if it doesn't exist
os.makedirs(processed_dir, exist_ok=True)

# Function to process a single XLS file
def process_xls(file_path):
    try:
        file_type = 'xls' if file_path.endswith('.xls') else 'xlsx'
        engine = 'xlrd' if file_type == 'xls' else 'openpyxl'
        df = pd.read_excel(file_path, sheet_name='Zeitreihen0h15', engine=engine)
        # Convert column names to strings
        df.columns = [str(col).strip() for col in df.columns]
        # Check for datetime column (e.g., Unnamed: 0)
        if 'Unnamed: 0' in df.columns:
            df['Timestamp'] = pd.to_datetime(df['Unnamed: 0'], errors='coerce')
            df = df.drop('Unnamed: 0', axis=1)
        # Combine date columns if they exist (DD, MM, YYYY, hh, min)
        date_cols = ['DD', 'MM', 'YYYY', 'hh', 'min']
        if all(col in df.columns for col in date_cols):
            df[date_cols] = df[date_cols].apply(pd.to_numeric, errors='coerce')
            df = df.dropna(subset=date_cols)
            df['Timestamp'] = pd.to_datetime(
                df[['YYYY', 'MM', 'DD', 'hh', 'min']].astype(str).agg('-'.join, axis=1),
                format='%Y-%m-%d-%H-%M',
                errors='coerce'
            )
            df = df.drop(date_cols, axis=1, errors='ignore')
        # Extract year from filename
        year = os.path.basename(file_path).split('-')[1][:4]
        df['Year'] = year
        # Convert numeric columns to float64
        numeric_cols = [col for col in df.columns if col not in ['Timestamp', 'Year']]
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        return df
    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {e}")
        return None

# 1. Read XLS files for 2009-2025 (fixed glob pattern)
energy_path = os.path.join(extract_dir, 'EDHD_Ch9_Dataset/swissGrid-dataset/Energy-overview-dataset')
energy_xls_files = sorted(glob.glob(os.path.join(energy_path, 'EnergieUebersichtCH-20*.xls')) + glob.glob(os.path.join(energy_path, 'EnergieUebersichtCH-20*.xlsx')))
energy_dfs = []
for f in energy_xls_files:
    df = process_xls(f)
    if df is not None:
        energy_dfs.append(df)

if energy_dfs:
    energy_overview_raw_timeseries = pd.concat(energy_dfs, ignore_index=True)
    # Sort by Timestamp if it exists
    if 'Timestamp' in energy_overview_raw_timeseries.columns:
        energy_overview_raw_timeseries = energy_overview_raw_timeseries.sort_values('Timestamp').reset_index(drop=True)
    # Save raw timeseries
    energy_overview_raw_timeseries.to_csv(os.path.join(processed_dir, 'energy_overview_raw_timeseries.csv'), index=False)
    print("energy_overview_raw_timeseries created with shape:", energy_overview_raw_timeseries.shape)
    print("Saved to:", os.path.join(processed_dir, 'energy_overview_raw_timeseries.csv'))
    print("Years covered:", sorted(energy_overview_raw_timeseries['Year'].unique()))
else:
    print("No valid Energy Overview XLS files processed (2009-2025).")
    energy_overview_raw_timeseries = pd.DataFrame()

# Function to clean the raw timeseries
def clean_energy_timeseries(df):
    # Drop unnamed or metadata columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed|anzMonate', case=False)]
    # Ensure Timestamp is datetime
    if 'Timestamp' in df.columns:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
        df = df.dropna(subset=['Timestamp'])
        df = df.sort_values('Timestamp').reset_index(drop=True)
    # Convert numeric columns
    numeric_cols = [col for col in df.columns if col not in ['Timestamp', 'Year']]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    # Drop fully NaN rows
    df = df.dropna(how='all')
    # Fill remaining NaNs with 0
    df = df.fillna(0)
    return df

# Clean energy_overview_raw_timeseries
if not energy_overview_raw_timeseries.empty:
    energy_overview_clean_timeseries = clean_energy_timeseries(energy_overview_raw_timeseries)
    energy_overview_clean_timeseries.to_csv(os.path.join(processed_dir, 'energy_overview_clean_timeseries.csv'), index=False)
    print("energy_overview_clean_timeseries created with shape:", energy_overview_clean_timeseries.shape)
    print("Saved to:", os.path.join(processed_dir, 'energy_overview_clean_timeseries.csv'))
else:
    print("No energy_overview_raw_timeseries to clean.")
    energy_overview_clean_timeseries = pd.DataFrame()

# Display information for clean timeseries
if not energy_overview_clean_timeseries.empty:
    print("\nColumn names in energy_overview_clean_timeseries:")
    print(energy_overview_clean_timeseries.columns.tolist())
    print("\nData description for energy_overview_clean_timeseries:")
    print(energy_overview_clean_timeseries.describe())
else:
    print("\nNo columns or description for energy_overview_clean_timeseries (empty).")

/tmp/ipykernel_306/476917588.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Timestamp'] = pd.to_datetime(df['Unnamed: 0'], errors='coerce')
/tmp/ipykernel_306/476917588.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Timestamp'] = pd.to_datetime(df['Unnamed: 0'], errors='coerce')
/tmp/ipykernel_306/476917588.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Timestamp'] = pd.to_datetime(df['Unnamed: 0'], errors='coerce')
/tmp/ipykernel_306/476917588.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back

energy_overview_raw_timeseries created with shape: (581389, 66)
Saved to: ../Processed_Data/energy_overview_raw_timeseries.csv
Years covered: ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']
energy_overview_clean_timeseries created with shape: (581372, 66)
Saved to: ../Processed_Data/energy_overview_clean_timeseries.csv

Column names in energy_overview_clean_timeseries:
['Summe endverbrauchte Energie Regelblock Schweiz\nTotal energy consumed by end users in the Swiss controlblock', 'Summe produzierte Energie Regelblock Schweiz\nTotal energy production Swiss controlblock', 'Summe verbrauchte Energie Regelblock Schweiz\nTotal energy consumption Swiss controlblock', 'Netto Ausspeisung aus dem Übertragungsnetz Schweiz\nNet outflow of the Swiss transmission grid', 'Vertikale Einspeisung ins Übertragungsnetz Schweiz\nGrid feed-in Swiss transmission grid', 'Positive Sekundär-Regelenergie\nPositive secundar

In [5]:
# Read, stack, clean, and save SwissGrid CSV data (2015-2025), display info

# Import necessary libraries
import pandas as pd
import glob
import os
import numpy as np
import re  # For year extraction

# Define directories
extract_dir = "../Extracted_Dataset"
processed_dir = "../Processed_Data"

# Create Processed_Data directory if it doesn't exist
os.makedirs(processed_dir, exist_ok=True)

# Function to process a single CSV file
def process_csv(file_path):
    try:
        # Read CSV with sep=';', header=0 (first row as headers)
        df = pd.read_csv(file_path, sep=';', low_memory=False)
        
        # Handle extra 'Land' column if present
        if 'Land' in df.columns:
            df = df.drop('Land', axis=1)
            print(f"Dropped 'Land' column in {os.path.basename(file_path)}")
        
        # Extract year from filename
        year_match = re.search(r'20\d{2}', os.path.basename(file_path))
        year = year_match.group() if year_match else 'Unknown'
        df['Year'] = year
        
        # Convert numeric columns (exclude text like 'Ausschreibung', 'Beschreibung', 'Year')
        text_cols = ['Ausschreibung', 'Beschreibung', 'Year']
        numeric_cols = [col for col in df.columns if col not in text_cols]
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        return df
    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {e}")
        return None

# 1. Read CSV files for 2015-2025
csv_path = os.path.join(extract_dir, 'EDHD_Ch9_Dataset/swissGrid-dataset/flexibility-market_power_need_in_switzerland/Archiv-Ergebnisse')
csv_files = sorted(glob.glob(os.path.join(csv_path, '*201[5-9]*.csv')) + glob.glob(os.path.join(csv_path, '*202*.csv')))
csv_dfs = []
for f in csv_files:
    df = process_csv(f)
    if df is not None:
        csv_dfs.append(df)

if csv_dfs:
    PRL_SRL_TRL_Ergebnis_raw_timeseries = pd.concat(csv_dfs, ignore_index=True)
    # Sort by Year or Ausschreibung if it exists
    if 'Ausschreibung' in PRL_SRL_TRL_Ergebnis_raw_timeseries.columns:
        PRL_SRL_TRL_Ergebnis_raw_timeseries = PRL_SRL_TRL_Ergebnis_raw_timeseries.sort_values('Ausschreibung').reset_index(drop=True)
    # Save raw timeseries
    PRL_SRL_TRL_Ergebnis_raw_timeseries.to_csv(os.path.join(processed_dir, 'PRL_SRL_TRL_Ergebnis_raw_timeseries.csv'), index=False)
    print("PRL_SRL_TRL_Ergebnis_raw_timeseries created with shape:", PRL_SRL_TRL_Ergebnis_raw_timeseries.shape)
    print("Saved to:", os.path.join(processed_dir, 'PRL_SRL_TRL_Ergebnis_raw_timeseries.csv'))
else:
    print("No valid SwissGrid CSV files processed (2015-2025).")
    PRL_SRL_TRL_Ergebnis_raw_timeseries = pd.DataFrame()

# Function to clean the raw timeseries
def clean_PRL_timeseries(df):
    # Drop unnamed or metadata columns if any
    df = df.loc[:, ~df.columns.str.contains('^Unnamed', case=False)]
    # Convert numeric columns
    text_cols = ['Ausschreibung', 'Beschreibung', 'Year']
    numeric_cols = [col for col in df.columns if col not in text_cols]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    # Drop fully NaN rows
    df = df.dropna(how='all')
    # Fill remaining NaNs with 0
    df = df.fillna(0)
    return df

# Clean PRL_SRL_TRL_Ergebnis_raw_timeseries
if not PRL_SRL_TRL_Ergebnis_raw_timeseries.empty:
    PRL_SRL_TRL_Ergebnis_clean_timeseries = clean_PRL_timeseries(PRL_SRL_TRL_Ergebnis_raw_timeseries)
    PRL_SRL_TRL_Ergebnis_clean_timeseries.to_csv(os.path.join(processed_dir, 'PRL_SRL_TRL_Ergebnis_clean_timeseries.csv'), index=False)
    print("PRL_SRL_TRL_Ergebnis_clean_timeseries created with shape:", PRL_SRL_TRL_Ergebnis_clean_timeseries.shape)
    print("Saved to:", os.path.join(processed_dir, 'PRL_SRL_TRL_Ergebnis_clean_timeseries.csv'))
else:
    print("No PRL_SRL_TRL_Ergebnis_raw_timeseries to clean.")
    PRL_SRL_TRL_Ergebnis_clean_timeseries = pd.DataFrame()

# Display information for clean timeseries
if not PRL_SRL_TRL_Ergebnis_clean_timeseries.empty:
    print("\nColumn names in PRL_SRL_TRL_Ergebnis_clean_timeseries:")
    print(PRL_SRL_TRL_Ergebnis_clean_timeseries.columns.tolist())
    print("\nData description for PRL_SRL_TRL_Ergebnis_clean_timeseries:")
    print(PRL_SRL_TRL_Ergebnis_clean_timeseries.describe())
else:
    print("\nNo columns or description for PRL_SRL_TRL_Ergebnis_clean_timeseries (empty).")


Dropped 'Land' column in 2017_PRL_SRL_TRL_Ergebnis.csv
Dropped 'Land' column in 2018-PRL-SRL-TRL-Ergebnis.csv
Dropped 'Land' column in 2019_PRL_SRL_TRL_Ergebnis.csv
Dropped 'Land' column in 2020_PRL_SRL_TRL_Ergebnis.csv
Dropped 'Land' column in 2021_PRL_SRL_TRL_Ergebnis.csv
Dropped 'Land' column in 2022-PRL-SRL-TRL-Ergebnis.csv
Dropped 'Land' column in 2023-PRL-SRL-TRL-Ergebnis.csv
Dropped 'Land' column in 20230906_regelleistung_2024_vorgezogene_beschaffung_ergebnis.csv
Dropped 'Land' column in 2024-PRL-SRL-TRL-Ergebnis.csv
Dropped 'Land' column in 2025-PRL-SRL-TRL-Ergebnis.csv
PRL_SRL_TRL_Ergebnis_raw_timeseries created with shape: (4587770, 16)
Saved to: ../Processed_Data/PRL_SRL_TRL_Ergebnis_raw_timeseries.csv
PRL_SRL_TRL_Ergebnis_clean_timeseries created with shape: (4587770, 16)
Saved to: ../Processed_Data/PRL_SRL_TRL_Ergebnis_clean_timeseries.csv

Column names in PRL_SRL_TRL_Ergebnis_clean_timeseries:
['Ausschreibung', 'Beschreibung', 'Angebotenes Volumen', 'Einheit', 'Zugesproch